In [2]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

In [10]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = '/home/leeyh/Downloads/Shanghai/'

In [4]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [6]:
model = CSRNet()

In [7]:
model = model.cuda()

In [35]:
checkpoint = torch.load('model_best.pth.tar')

In [36]:
model.load_state_dict(checkpoint['state_dict'])

In [37]:
mae = 0
for i in xrange(len(img_paths)):
    img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print i,mae
print mae/len(img_paths)

0 19.4618225098
1 56.0270690918
2 237.42098999
3 250.008789062
4 277.621917725
5 292.61365509
6 431.04094696
7 621.934074402
8 626.175041199
9 683.771781921
10 728.903190613
11 988.464836121
12 1072.35503387
13 1086.75745392
14 1139.51770782
15 1187.12958527
16 1244.76972198
17 1357.30127716
18 1499.7251358
19 1531.53779602
20 1564.71266174
21 1741.81813049
22 1799.57826233
23 1856.79542542
24 1997.93318176
25 2001.67614746
26 2056.48977661
27 2094.64889526
28 2146.35804749
29 2298.53785706
30 2316.62382507
31 2353.09817505
32 2409.85276794
33 2575.56442261
34 2594.92245483
35 2610.66140747
36 2621.21057129
37 2629.99487305
38 2740.87393188
39 2842.61392212
40 2846.01530457
41 2906.07719421
42 2927.62335968
43 2987.65406036
44 3014.2397995
45 3143.8065567
46 3825.0911026
47 3854.01618195
48 3854.2317276
49 3871.20517731
50 3919.21517181
51 4017.22032928
52 4080.6140976
53 4200.34767914
54 4204.83093262
55 4231.756073
56 4265.32553101
57 4424.53753662
58 4475.64712524
59 4498.09152222
6